In [1]:
# set conda
!wget -c https://repo.continuum.io/miniconda/Miniconda3-latest-Linux-x86_64.sh
!chmod +x Miniconda3-latest-Linux-x86_64.sh
!bash ./Miniconda3-latest-Linux-x86_64.sh -b -f -p /usr/local

# install psi4
!conda install -y psi4 python=3.7 -c psi4

# set path
import sys
sys.path.append("/usr/local/lib/python3.7/site-packages/")

--2022-04-19 01:04:21--  https://repo.continuum.io/miniconda/Miniconda3-latest-Linux-x86_64.sh
Resolving repo.continuum.io (repo.continuum.io)... 104.18.201.79, 104.18.200.79, 2606:4700::6812:c84f, ...
Connecting to repo.continuum.io (repo.continuum.io)|104.18.201.79|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: https://repo.anaconda.com/miniconda/Miniconda3-latest-Linux-x86_64.sh [following]
--2022-04-19 01:04:21--  https://repo.anaconda.com/miniconda/Miniconda3-latest-Linux-x86_64.sh
Resolving repo.anaconda.com (repo.anaconda.com)... 104.16.131.3, 104.16.130.3, 2606:4700::6810:8203, ...
Connecting to repo.anaconda.com (repo.anaconda.com)|104.16.131.3|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 75660608 (72M) [application/x-sh]
Saving to: ‘Miniconda3-latest-Linux-x86_64.sh’

Miniconda3-latest-L 100%[===================>]  72.16M   107MB/s    in 0.7s    

2022-04-19 01:04:22 (107 MB/s) - ‘Miniconda3-latest-Linux

In [2]:
# import Psi4
import psi4
# check Psi4 version
print(psi4.__version__)

1.5


In [3]:
import time
import numpy as np

mol: psi4.core.Molecule = psi4.geometry('''
0 1
O        0.000000    0.695000   -0.092486
O       -0.000000   -0.695000   -0.092486
H       -0.388142    0.895249    0.739888
H        0.388142   -0.895249    0.739888
''')

scan_range = np.linspace(2.5, 1.1, 10)
level = 'HF/3-21G'
energies = []
structures = []

for dist in scan_range:
    psi4.set_options({'geom_maxiter': 200,
                      'full_hess_every': 0,
                      'optking__fixed_distance': f'1 2 {dist}',
                      'optking__opt_coordinates': 'both',
                      'optking__interfrag_step_limit': 0.3,
                      'optking__intrafrag_step_limit': 0.3
                      })
    print('-----')
    print(f'optimization at {dist:.2f}')
    start = time.time()
# If optimization fails, start over
    try:
        energy = psi4.optimize(level, molecule=mol)
    except psi4.driver.p4util.exceptions.OptimizationConvergenceError as err:
        print(err)
        psi4.set_options({'full_hess_every': 1})
        energy = psi4.optimize(level, molecule=mol)
    end = time.time()
    print(f'It took {(end - start)/60:.2f} min for the optimization.')
    print(f'energy: {energy:.4f} a.u.')
    print(f'optimized structure:\n{mol.save_string_xyz()}')
    energies.append(energy)
    structures.append(mol.save_string_xyz())

energies = np.array(energies)
print(energies)
# Get the structure that maximizes energy
max_id = np.argmax(energies)
print(f'energy is maximized at {scan_range[max_id]:.2f} A.')
print(f'structure:\n{structures[max_id]}')
with open('ts.xyz', 'w') as f:
    print(mol.natom(), file=f)
    print(structures[max_id], file=f)

-----
optimization at 2.50
Could not converge geometry optimization in 200 iterations.
Optimizer: Optimization complete!
It took 3.18 min for the optimization.
energy: -149.7428 a.u.
optimized structure:
0 1
 O    0.166983242512    1.237422841275   -0.000091339652
 O   -0.166983242512   -1.237422841275   -0.000091339652
 H   -0.811994630859    1.270242469274    0.001449626553
 H    0.811994630859   -1.270242469274    0.001449626553

-----
optimization at 2.34
Could not converge geometry optimization in 200 iterations.
Optimizer: Optimization complete!
It took 1.49 min for the optimization.
energy: -149.7728 a.u.
optimized structure:
0 1
 O    1.158107898715   -0.168571533151   -0.000036276147
 O   -1.158107898715    0.168571533151   -0.000036276147
 H    1.227783608774    0.807584464396    0.000575728785
 H   -1.227783608774   -0.807584464396    0.000575728785

-----
optimization at 2.19
Could not converge geometry optimization in 200 iterations.
Optimizer: Optimization complete!
It to